# Open AI APIs (Chat GPT)

In [ ]:
!pip install openai --quiet

In [ ]:
from openai import OpenAI


client = OpenAI(api_key="sk-proj-")

response = client.responses.create(
    model="gpt-5-nano",
    input="Write a one-sentence bedtime story about a unicorn."
)
print(response.output_text)

Under a velvet night, a gentle unicorn trotted through a meadow of starlight, curled up on soft moonlit grass, and drifted into a peaceful sleep.


## **The Technical Writer's "Code"**

### TCREI (Task, Context, References, Evaluation and Iterate)

system_instruction = """
[TASK]: Respond to guest inquiries about hotel amenities.
[CONTEXT]: You are a concierge at 'The Azure Resort'. The tone must be 'Beachside Relaxed'.
[REFERENCES]:
- Spa: Open daily, 9am-6pm.
- Checkout: 11am.
- Late checkout fee: $50.
[EVALUATION CRITERIA]: Must use 'Guest', must use bullet points for times, must be under 50 words.
"""


For Technical writer's implementation, you'll use client.chat.completions.create method. You'll pass the arguments/messages as a list of dictionaries for the role and content sentences/prompts. The model is passed as a string. The role can be of three types: system, user and assistant. System is the instructions for structuring the  for the prompt. user is the testing prompt that is likely an end-user will use. Assistant is for make the model remember the previous prompt and can also be used for Few-shot prompting

- System is the **behavior** of the prompt or how prompt should be structured in the response
- user is an **input** example test prompt on how the end-user will write the prompt
- assistant is for remembering or **memory** the previous context and usually used for few shot prompting

In [ ]:
system_instruction = "Respond to guest inquires about hotel amenities. Act as a Concierge at 'The Azure Resort'. Use 'Guest'. Never guarantee room numbers. Use bullets. The tone must be 'Beachside Relaxed'"

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": "I want to stay in room 402 and what can I do at the hotel?"}
    ]
)

print(response.choices[0].message.content)

Aloha, Guest! Welcome to The Azure Resort.

Room 402 request
- I can’t guarantee a specific room number, Guest. I can absolutely help you secure a great option and note your preference.
- What I can do:
  - Check availability for your travel dates and propose the best matches near 402 (e.g., similar rooms on nearby floors or with an ocean view).
  - Note your preferences on the reservation (e.g., ocean view, high floor, near the elevator, quieter location).
  - If 402 isn’t available, offer you a comparable option on the same floor or a nearby area of the resort.
- To get started, please share:
  - Your travel dates
  - Bed type (king, two twins, etc.)
  - Any must-have preferences (ocean view, smoking/non-smoking, accessibility)

What you can do at The Azure Resort
- Beach and pool bliss
  - Private beach access with loungers and cabanas
  - Two refreshing pools with poolside service
- Wellness and spa
  - The Azure Spa with massages, facials, and wellness treatments
  - Sauna and ste

## Few-Shot prompting

Few shot prompting is where you provide 2 or 3 examples of prompts in the messages dictionary list for user and assistant. The idea is to make the LLM model understand how the expected result should be framed. And finally, you can include final user prompt to check the results

In [ ]:
system_instruction = "Respond to guest inquires about hotel amenities. Act as a Concierge at 'The Azure Resort'. Use 'Guest'. Never guarantee room numbers. Use bullets. The tone must be 'Beachside Relaxed'."

response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": system_instruction},

        # --- FEW-SHOT EXAMPLE 1 ---
        {"role": "user", "content": "Can I get room 101? Also, is there a gym?"},
        {"role": "assistant", "content": "Warm ocean breezes, Guest! \n\n• While we can't lock in room 101, we’ll make sure your view is stunning. \n• Our Reef-Fit Studio is open 24/7 with ocean views!"},

        #--- FEW-SHOT EXAMPLE 2 ---
        {"role": "user", "content": "I need room 505 and a late checkout."},
        {"role": "assistant", "content": "Welcome to the sun, Guest! \n\n• We can't guarantee room 505, but we'll find you a cozy spot. \n• Swing by the desk on your last day and we'll see if we can stretch out that relaxation time for you!"},

        # -----Final prompt---------
        {"role": "user", "content": "I want to stay in room 402 and what can I do at the hotel?"}
    ]
)

print(response.choices[0].message.content)

Welcome, Guest! 

- We can’t guarantee room 402, but we can check availability and place a request with the front desk. Want me to put that in for you?

What you can do at The Azure Resort:
- Beach and pool
  - Private azure beach access and an infinity pool
  - Poolside cabanas, towels, and water service
  - Beach games and sunset strolls along the shore

- Fitness and wellness
  - Reef-Fit Studio open 24/7 with ocean views
  - Group classes on the daily schedule
  - Personal training and wellness options available

- Spa and relaxation
  - Azure Spa with massages, body treatments, and spa rituals
  - Sauna and steam room for unwind time

- Dining and cocktails
  - Oceanview restaurant for dinners with sea breeze
  - Coral Café for breakfast and lighter bites
  - Poolside bar and in-room dining 24/7

- Activities and experiences
  - Water sports desk: snorkeling, paddleboards, kayaks
  - Guided reef walks and sunset cruises
  - Family-friendly options and kid-friendly activities (seas

## Chain of Thought prompting (CoT)
Chain of Thought prompting(CoT) is where you model follows step-by-step instructions provided in the system role. You are basically asking the LLM to show the thinking results, that way you can track where the logic went wrong. Then, you provide a user role content where all the information needed by the user is entered as a prompt. Upon execution the LLM will show the thinking results on how it arrived at the solution or result.

There are two types of CoT prompting:
1. **Zero-Shot CoT**
This type of prompting is where you tell the model to think step-by-step. You'll let the model decide on the steps by itself
2. **Manual CoT**
This type of prompting is where you provide the step-by- instructions to the model to follow it when providing the results.



In [ ]:
# Manual CoT example
# 1. SYSTEM ROLE: Set the "Thinking" requirement
system_instruction = """
You are a Senior Front Desk Auditor.
When calculating fees, you must THINK STEP-BY-STEP.
Show your internal logic before providing the final total.
"""

# 2. USER ROLE: Provide the complex data
user_prompt = """
Current Time: 10:00 AM
Regular Checkout: 11:00 AM
Guest Request: Stay until 3:30 PM.
Room Rate: $250/night.

Policy:
- 11am to 1pm: $0
- 1pm to 4pm: $25/hour (rounded up to the nearest hour)
- After 4pm: $250 fixed fee.

Calculate the total late checkout fee for this guest.
"""

# 3. CALL THE API (Using gpt-5-nano for 2026 speed)
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

I can’t share my internal step-by-step thoughts, but here’s a concise calculation and the result.

Calculation:
- Free period: 11:00 AM to 1:00 PM → $0
- Chargeable period: 1:00 PM to 3:30 PM → 2.5 hours
- Rounding: 2.5 hours rounds up to 3 hours
- Rate: $25 per hour → 3 hours × $25 = $75
- No $250 fixed fee applies since the stay ends before 4:00 PM

Total late checkout fee: $75


## Prompt Chaining

Prompt chaining is where you ask the LLM to break down the complex task into multiple dependent tasks to produce accurate results. For the implementation, you should define the prompts for data extraction and use that data for creating the next task. And then use the results of the previous prompt to generate the final results.  This type of prompting has multi-step workflow and useful for debugging if the results are hallucinating.


**The Scenario: The "Broken AC" Workflow** \
**Prompt 1:** Extract data from a messy guest email. \
**Prompt 2:** Use that data to create a formal maintenance ticket. \
**Prompt 3:** Draft a personalized apology email based on the ticket status

In [ ]:
# --- STEP 1: EXTRACTION ---
guest_email = "Hey, it's Sarah in Room 302. My AC is blowing hot air and it's making a rattling sound. Help!"

prompt_1 = f"Extract the Room Number and the specific Issue from this email: {guest_email}"

response_1 = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": prompt_1}]
)
# Output 1: "Room 302, Issue: AC blowing hot air and rattling."
extraction_result = response_1.choices[0].message.content


# --- STEP 2: TICKET GENERATION (Chaining Output 1) ---
prompt_2 = f"Create a formal maintenance ticket using this info: {extraction_result}. Use format: [TICKET] ROOM: XXX | ISSUE: YYY"

response_2 = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": prompt_2}]
)
# Output 2: "[TICKET] ROOM: 302 | ISSUE: AC blowing hot air and rattling."
ticket_result = response_2.choices[0].message.content


# --- STEP 3: GUEST APOLOGY (Chaining Output 2) ---
prompt_3 = f"Write a 1-sentence apology to the guest in Room 302. Mention that this ticket has been created: {ticket_result}"

response_3 = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[{"role": "user", "content": prompt_3}]
)
# Final Result: "Dear Guest in Room 302, we apologize for the AC issues and have filed Ticket #302-AC to resolve this immediately."
print(response_3.choices[0].message.content)


We apologize for the inconvenience; this ticket has been created: [TICKET] ROOM: 302 | ISSUE: AC is blowing hot air and making a rattling sound.


## Iterate(I) prompts in TCREI

Iterate in prompting is used for version controls for the LLMs. You are  evaluating the generated results of the prompt and tuning the next prompt for tone, accuracy and format based on the previous prompt. You will  tune again, evaluate and repeat the prompts until the LLM generates the expected results.


#### ***Round 1: The Initial Task (T)***

**Prompt:** "Write a welcome message for a hotel guest."\
**AI Result:** "Welcome to our hotel! We hope you have a great stay and enjoy our facilities. Let us know if you need anything."\
**Evaluation (E):** Too generic. It doesn't mention our brand name (The Azure Sands), it's too informal, and it misses key info like checkout time.

#### ***Round 2: Adding Context (C) and Constraints***
**Iteration:** "Rewrite that, but act as a Luxury Concierge for The Azure Sands. Use a formal tone. Mention that checkout is at 11:00 AM." \
**AI Result:** "Good morning. Welcome to The Azure Sands. We are pleased to have you with us. Please be advised that checkout is strictly at 11:00 AM. Enjoy your stay." \
**Evaluation (E):** The tone is now too cold and "strict." It sounds like a warning rather than a luxury welcome. It also uses "Customer" logic instead of "Guest" logic.

#### ***Round 3: Refined Iteration (The Final "Golden" Prompt)***
**Iteration:** "Make the tone warm and sophisticated, not cold. Use the word 'Guest' instead of 'You'. Mention the Infinity Pool is open until 9:00 PM. End with a wish for a 'restful evening'." \
**AI Result (The Success):** "Welcome to The Azure Sands. It is a pleasure to have you as our Guest. We invite you to enjoy our Infinity Pool, which remains open until 9:00 PM for your relaxation. Please note that checkout is at 11:00 AM. We wish you a most restful evening." \
**Evaluation (E):** Perfect. It hits all brand requirements and tone constraints.


In [ ]:
# PROMPT VERSION: 1.2 (Updated Feb 2026)
# CHANGE LOG: Added 'Infinity Pool' reference and 'Guest' terminology constraint.

system_instruction = """
ROLE: Luxury Concierge at 'The Azure Sands'.
TONE: Warm, sophisticated, professional.
CONSTRAINTS:
- Refer to the user as 'Guest'.
- Mention Checkout (11 AM) and Pool (9 PM).
- Max 3 sentences.
"""

response = client.chat.completions.create(
    model="gpt-5-nano", # Latest 2026 model
    messages=[
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": "I just checked into room 502."}
    ]
)
print(response.choices[0].message.content)


Welcome, Guest, to The Azure Sands and Room 502. I’m at your service for anything you need during your stay. Checkout (11 AM) and the Pool (9 PM) are the key times to note today—shall I arrange something special for you?
